In [1]:
from diffusers import UNet2DModel, DDPMPipeline

In [2]:
image_pipe = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256")
repo_id = "google/ddpm-church-256"
model = UNet2DModel.from_pretrained(repo_id)

random_model = UNet2DModel()

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/455M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/790 [00:00<?, ?B/s]

In [3]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 128  # the generated image resolution
    train_batch_size = 16
    eval_batch_size = 16  # how many images to sample during evaluation
    num_epochs = 50
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = 'no'  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = 'ddpm-fashionmnist-128'  # the model namy locally and on the HF Hub

    push_to_hub = True  # whether to upload the saved model to the HF Hub
    hub_private_repo = False  
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 0

config = TrainingConfig()

In [4]:
from datasets import load_dataset

# load dataset from the hub
dataset = load_dataset("fashion_mnist")
image_size = 28
channels = 1
batch_size = 128

from torchvision import transforms

preprocess = transforms.Compose(
    [
        transforms.Resize((config.image_size, config.image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)


def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    return {"images": images}

dataset.set_transform(transform)

import torch

train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

from diffusers.optimization import get_cosine_schedule_with_warmup

lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

from diffusers import DDPMPipeline

import math

def make_grid(images, rows, cols):
    w, h = images[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    for i, image in enumerate(images):
        grid.paste(image, box=(i%cols*w, i//cols*h))
    return grid

def evaluate(config, epoch, pipeline):
    # Sample some images from random noise (this is the backward diffusion process).
    # The default pipeline output type is `List[PIL.Image]`
    images = pipeline(
        batch_size = config.eval_batch_size, 
        generator=torch.manual_seed(config.seed),
    ).sample

    # Make a grid out of the images
    image_grid = make_grid(images, rows=4, cols=4)

    # Save the images
    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")
    
from diffusers import DDPMScheduler

noise_scheduler = DDPMScheduler(num_train_timesteps=1000, tensor_format="pt")

In [ ]:
from accelerate import Accelerator
from diffusers.hub_utils import init_git_repo, push_to_hub

from tqdm.auto import tqdm
import os

def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize accelerator and tensorboard logging
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps, 
        log_with="tensorboard",
        logging_dir=os.path.join(config.output_dir, "logs")
    )
    if accelerator.is_main_process:
        if config.push_to_hub:
            repo = init_git_repo(config, at_init=True)
        accelerator.init_trackers("train_example")
    
    # Prepare everything
    # There is no specific order to remember, you just need to unpack the 
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )
    
    global_step = 0

    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch['images']
            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape).to(clean_images.device)
            bs = clean_images.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (bs,), device=clean_images.device).long()

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)
            
            with accelerator.accumulate(model):
                # Predict the noise residual
                # noise_pred = predict_v(model(noisy_images, timesteps, return_dict=False)[0], eps, logsnr)
                noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
            
            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        # After each epoch you optionally sample some demo images with evaluate() and save the model
        if accelerator.is_main_process:
            pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                evaluate(config, epoch, pipeline)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                if config.push_to_hub:
                    push_to_hub(config, pipeline, repo, commit_message=f"Epoch {epoch}", blocking=True)
                else:
                    pipeline.save_pretrained(config.output_dir) 

In [7]:
from accelerate import notebook_launcher
args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)

Launching training on CPU.


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/y3/17f2rsd52z9_kgw7n7_54y1r0000gn/T/ipykernel_30946/2823307728.py:4 in       │
│ <module>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/y3/17f2rsd52z9_kgw7n7_54y1r0000gn/T/ipykernel_30946/2823307728.py'          │
│ /Users/liviacavalcanti/dd/diffusers/v_diffusers/lib/python3.9/site-packages/accelerate/la │
│ unchers.py:134 in notebook_launcher                                                       │
│                                                                                           │
│   131 │   │   │   else:                                                                   │
│   132 │   │   │   │   print("Launching training on CPU.")                                 │
│   133 │   │   │   with patch_environment(use_mps_device=use_mps_device):                  │
│ ❱ 134 │   │   │   │   function(*args)                                                     │
│   135                                                                                     │
│   136                                                                                     │
│   137 def debug_launcher(function, args=(), num_processes=2):                             │
│                                                                                           │
│ /var/folders/y3/17f2rsd52z9_kgw7n7_54y1r0000gn/T/ipykernel_30946/3855608178.py:17 in      │
│ train_loop                                                                                │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/y3/17f2rsd52z9_kgw7n7_54y1r0000gn/T/ipykernel_30946/3855608178.py'          │
│                                                                                           │
│ /Users/liviacavalcanti/dd/diffusers/src/diffusers/hub_utils.py:65 in init_git_repo        │
│                                                                                           │
│    62 │   else:                                                                           │
│    63 │   │   repo_name = args.hub_model_id                                               │
│    64 │   if "/" not in repo_name:                                                        │
│ ❱  65 │   │   repo_name = get_full_repo_name(repo_name, token=hub_token)                  │
│    66 │                                                                                   │
│    67 │   try:                                                                            │
│    68 │   │   repo = Repository(                                                          │
│                                                                                           │
│ /Users/liviacavalcanti/dd/diffusers/src/diffusers/hub_utils.py:42 in get_full_repo_name   │
│                                                                                           │
│    39 │   if token is None:                                                               │
│    40 │   │   token = HfFolder.get_token()                                                │
│    41 │   if organization is None:                                                        │
│ ❱  42 │   │   username = whoami(token)["name"]                                            │
│    43 │   │   return f"{username}/{model_id}"                                             │
│    44 │   else:                                                                           │
│    45 │   │   return f"{organization}/{model_id}" 

In [ ]:
def predict_v(*, x, eps, logsnr):
  logsnr = utils.broadcast_from_left(logsnr, x.shape)
  alpha_t = jnp.sqrt(jax.nn.sigmoid(logsnr))
  sigma_t = jnp.sqrt(jax.nn.sigmoid(-logsnr))
  return alpha_t * eps - sigma_t * x